In [ ]:
import os
import xml.etree.ElementTree as ET

def process_xml(xml_file):
    try:
        tree = ET.parse(xml_file)
        root = tree.getroot()

        # Extract image details
        filename = root.find('filename').text
        size = root.find('size')
        width = int(size.find('width').text)
        height = int(size.find('height').text)

        image_info = {
            'filename': filename,
            'width': width,
            'height': height,
            'objects': []
        }

        for obj in root.iter('object'):
            obj_dict = {}
            obj_dict['name'] = obj.find('name').text
            bbox = obj.find('bndbox')
            obj_dict['coordinates'] = [int(bbox.find('xmin').text),
                                       int(bbox.find('ymin').text),
                                       int(bbox.find('xmax').text),
                                       int(bbox.find('ymax').text)]
            image_info['objects'].append(obj_dict)

        return image_info

    except ET.ParseError as e:
        print(f"Error parsing file {xml_file}: {e}")
        return None

def process_all_xml(directory):
    all_data = []
    for file in os.listdir(directory):
        if file.endswith('.xml'):
            file_path = os.path.join(directory, file)
            data = process_xml(file_path)
            if data:
                all_data.append(data)

    return all_data

# Example usage
directory_path = '/content/drive/MyDrive/est/xml'
all_data = process_all_xml(directory_path)
print(all_data)


Error parsing file /content/drive/MyDrive/est/xml/2.xml: mismatched tag: line 1, column 323
[{'filename': '18.jpg', 'width': 448, 'height': 448, 'objects': [{'name': 'coin_1', 'coordinates': [81, 287, 81, 287]}, {'name': 'coin_2', 'coordinates': [28, 196, 28, 196]}, {'name': 'coin_3', 'coordinates': [367, 279, 367, 279]}, {'name': 'coin_4', 'coordinates': [423, 194, 423, 194]}]}, {'filename': '1.jpg', 'width': 448, 'height': 448, 'objects': [{'name': 'coin_1', 'coordinates': [89, 167, 89, 167]}, {'name': 'coin_2', 'coordinates': [93, 93, 93, 93]}, {'name': 'coin_3', 'coordinates': [346, 171, 346, 171]}, {'name': 'coin_4', 'coordinates': [336, 89, 336, 89]}]}, {'filename': '11.jpg', 'width': 448, 'height': 448, 'objects': [{'name': 'coin_1', 'coordinates': [69, 340, 69, 340]}, {'name': 'coin_2', 'coordinates': [31, 213, 31, 213]}, {'name': 'coin_3', 'coordinates': [390, 270, 390, 270]}, {'name': 'coin_4', 'coordinates': [434, 162, 434, 162]}]}, {'filename': '10.jpg', 'width': 448, 'heig

In [ ]:
# Exemple d'utilisation
transformations = [{'type': 'flip_horizontal'},{'type': 'flip_vertical'},{'type': 'rotate', 'angle': 90}]
chemin_repertoire_entree = '/content/drive/MyDrive/est/image/'
chemin_repertoire_sortie = '/content/drive/MyDrive/est/da'


In [ ]:
import cv2
import os

def flip_horizontal(bbox, image_width):
    x_min, y_min= bbox
    return  [image_width - x_min, y_min]

# Assumons que les fonctions de transformation sont définies ici

def appliquer_transformations_et_enregistrer(all_data, repertoire_images, repertoire_sortie, transformations):
    # Créer le répertoire de sortie s'il n'existe pas
    if not os.path.exists(repertoire_sortie):
        os.makedirs(repertoire_sortie)

    for data in all_data:
        nom_fichier_original = data['filename']
        chemin_image = os.path.join(repertoire_images, nom_fichier_original)
        image = cv2.imread(chemin_image)

        if image is None:
            print(f"Impossible de lire l'image : {nom_fichier_original}")
            continue

        # Appliquer les transformations à l'image
        image_transformee = image
        for transformation in transformations:
            if transformation['type'] == 'flip_horizontal':
                image_transformee = cv2.flip(image_transformee, 1)
                sufixe = 'horizontal'
            # Ajouter d'autres transformations ici

        # Mettre à jour le nom de fichier dans all_data et sauvegarder l'image transformée
        base_nom_fichier, ext_nom_fichier = os.path.splitext(nom_fichier_original)
        nouveau_nom_fichier = f"{base_nom_fichier}{sufixe}{ext_nom_fichier}"
        data['filename'] = nouveau_nom_fichier
        cv2.imwrite(os.path.join(repertoire_sortie, nouveau_nom_fichier), image_transformee)

        # Mettre à jour les coordonnées dans all_data
        for obj in data['objects']:
            bbox = obj['coordinates']
            for transformation in transformations:
                if transformation['type'] == 'flip_horizontal':
                    bbox = flip_horizontal(bbox, image.shape[1])
                # Ajouter d'autres mises à jour de transformation ici
            obj['coordinates'] = bbox

    return all_data






In [ ]:
all_data_transf = appliquer_transformations_et_enregistrer(all_data, chemin_repertoire_entree, chemin_repertoire_sortie, transformations)

In [ ]:
import cv2
import os

def flip_vertical(bbox, image_height):
    x_min, y_min= bbox
    return  [x_min, image_height - y_min]

# Assumons que les fonctions de transformation sont définies ici

def appliquer_transformations(data, repertoire_images, repertoire_sortie, transformations):
    # Créer le répertoire de sortie s'il n'existe pas
    if not os.path.exists(repertoire_sortie):
        os.makedirs(repertoire_sortie)

    for data in all_data:
        nom_fichier_original = data['filename']
        chemin_image = os.path.join(repertoire_images, nom_fichier_original)
        image = cv2.imread(chemin_image)

        if image is None:
            print(f"Impossible de lire l'image : {nom_fichier_original}")
            continue

        # Appliquer les transformations à l'image
        image_transformee = image
        for transformation in transformations:
            if transformation['type'] == 'flip_vertical':
                image_transformee = cv2.flip(image_transformee, 0)
                sufixe = 'vertical'
            # Ajouter d'autres transformations ici

        # Mettre à jour le nom de fichier dans all_data et sauvegarder l'image transformée
        base_nom_fichier, ext_nom_fichier = os.path.splitext(nom_fichier_original)
        nouveau_nom_fichier = f"{base_nom_fichier}{sufixe}{ext_nom_fichier}"
        data['filename'] = nouveau_nom_fichier
        cv2.imwrite(os.path.join(repertoire_sortie, nouveau_nom_fichier), image_transformee)

        # Mettre à jour les coordonnées dans all_data
        for obj in data['objects']:
            bbox = obj['coordinates']
            for transformation in transformations:
                if transformation['type'] == 'flip_vertical':
                    bbox = flip_vertical(bbox, image.shape[1])
                # Ajouter d'autres mises à jour de transformation ici
            obj['coordinates'] = bbox

    return all_data


In [ ]:
all_data_transf_ver = appliquer_transformations(all_data, chemin_repertoire_entree, chemin_repertoire_sortie, transformations)

In [ ]:
import math
import numpy as np

def rotate_bbox(bbox, angle_degrees, image_center):
    x_min, y_min, x_max, y_max = bbox
    corners = [(x_min, y_min), (x_max, y_min), (x_min, y_max), (x_max, y_max)]

    angle_radians = math.radians(angle_degrees)
    rotated_corners = [
        (
            math.cos(angle_radians) * (x - image_center[0]) - math.sin(angle_radians) * (y - image_center[1]) + image_center[0],
            math.sin(angle_radians) * (x - image_center[0]) + math.cos(angle_radians) * (y - image_center[1]) + image_center[1]
        )
        for x, y in corners
    ]

    # Calculate the new bounding box from the rotated corners
    xs, ys = zip(*rotated_corners)
    new_x_min, new_x_max = min(xs), max(xs)
    new_y_min, new_y_max = min(ys), max(ys)

    return [int(new_x_min), int(new_y_min), int(new_x_max), int(new_y_max)]


def rotate_image(image, angle):
    # Load the image

    if image is None:
        print(f"Error: Unable to load image at {image}")
        return

    # Get the image dimensions
    (h, w) = image.shape[:2]

    # Calculate the center of the image
    center = (w / 2, h / 2)

    # Calculate the rotation matrix
    M = cv2.getRotationMatrix2D(center, angle, 1.0)

    # Perform the rotation
    rotated = cv2.warpAffine(image, M, (w, h))
    return rotated

In [ ]:
def appliquer_transformations(data, repertoire_images, repertoire_sortie, transformations):
    if not os.path.exists(repertoire_sortie):
        os.makedirs(repertoire_sortie)

    for data in all_data:
        nom_fichier_original = data['filename']
        chemin_image = os.path.join(repertoire_images, nom_fichier_original)
        image = cv2.imread(chemin_image)

        if image is None:
            print(f"Impossible de lire l'image : {nom_fichier_original}")
            continue

        image_center = (image.shape[1] // 2, image.shape[0] // 2)
        suffixe = ''

        for transformation in transformations:
            if transformation['type'] == 'rotate':
                angle = transformation.get('angle', 0)
                image = rotate_image(image, angle)
                suffixe += f'_rotate_{angle}'

                # Update the bounding boxes
                for obj in data['objects']:
                    bbox = obj['coordinates']
                    obj['coordinates'] = rotate_bbox(bbox, angle, image_center)

        # Save the transformed image and update the filename
        base_nom_fichier, ext_nom_fichier = os.path.splitext(nom_fichier_original)
        nouveau_nom_fichier = f"{base_nom_fichier}{suffixe}{ext_nom_fichier}"
        cv2.imwrite(os.path.join(repertoire_sortie, nouveau_nom_fichier), image)
        data['filename'] = nouveau_nom_fichier

    return all_data


In [ ]:
all_data_transf_ro = appliquer_transformations(all_data, chemin_repertoire_entree, chemin_repertoire_sortie, transformations)

In [ ]:
for obj in data["objects"]:
    obj["coordinates"] = obj["coordinates"][:2]

In [ ]:
import json
with open('/content/drive/MyDrive/updated_image_data.json', 'r') as file:
    data = json.load(file)


In [ ]:
all = data + all_data_transf_ro

In [ ]:
import json

# Assuming 'data_list' has been modified
with open('/content/drive/MyDrive/updated_image_data.json', 'w') as file:
    json.dump(all, file, indent=4)




In [ ]:
import shutil
import os

# Define your directories
folder1 = '/content/drive/MyDrive/est/da'
folder2 = '/content/drive/MyDrive/est/dataset'
target_folder = '/content/drive/MyDrive/est/data'

# Create the target directory if it doesn't exist
if not os.path.exists(target_folder):
    os.makedirs(target_folder)

# Function to copy images from a source folder to the target folder
def copy_images(source_folder, target_folder):
    for filename in os.listdir(source_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff')):
            file_path = os.path.join(source_folder, filename)
            shutil.copy(file_path, target_folder)

# Copy images from both folders to the target folder
copy_images(folder1, target_folder)
copy_images(folder2, target_folder)
